1. Neither model underfit, Both hit strong training performance quickly and validation macro-F1 reaches 0.70+, so capacity + optimization are sufficient. 
Both models show mild–moderate overfitting after the mid-training point. For MLP, the train Macro F1 keep climbing while the validation Macro F1 plateau around 0.55 and wobbles. Train loss steadily decreases, but val loss stops improving and becomes noisy/spiky. For LSTM, the train loss decreces steadily, but the validation loss fluctuates fiercely while validation Macro F1 ends up good. 

With this dataset’s heavy class imbalance (neutral dominates), an unweighted model tends to optimize accuracy by overpredicting neutral. Using class_weight pushes the model to treat minority errors as expensive, which typically improves macro-F1 materially and increases minority-class recall/precision balance. Class weights increase the gradient contribution from rare classes. That tends to make training less smooth (more variance batch-to-batch) and make validation loss spikier, because a few confidently-wrong minority examples can dominate cross-entropy. Class weights likely raised the macro-F1 to the required level.

2. LSTM generalized slightly better. Evidence:
Test Macro-F1: LSTM 0.7161 vs MLP 0.7115 (primary metric).
LSTM: val macro-F1 0.7097 → test 0.7161 (+0.0064)
MLP: val macro-F1 0.7202 → test 0.7115 (−0.0087)
The LSTM’s test performance matches (and slightly exceeds) its validation performance, while the MLP drops a bit from validation to test.
In terms of accuracy, both are close, but LSTM edges out on test macro-F1.


Positive was misclassified the most in both models, mainly as neutral.

From the confusion matrices:
LSTM (true positive row): positive → neutral = 67, positive → negative = 25, correct = 112
Misclassified positives = 92 out of 204, 45% error rate
MLP (true positive row): positive → neutral = 68, positive → negative = 23, correct = 113
Misclassified positives =  91 out of  204, 45% error rate
Negatives and neutrals had much lower error rates.

Even with class weights, the model still sees many more neutral samples, so decision boundaries often tilt toward predicting neutral when the signal is weak. Financial writing frequently frames good news cautiously (“slightly improved”, “expects growth”, “could increase”), which looks semantically close to neutral. Even “50agree” still includes sentences where annotators weren’t unanimous; borderline “neutral vs positive” is exactly where disagreement is common, so the model’s errors concentrate there.

3. Mean-pooling turns a sentence into a bag-of-embeddings:
Loses word order and syntax: “profits fell despite revenue growth” and “profits grew despite revenue fall” can end up close after averaging.
Weak on negation/contrast cues (“not”, “but”) because those cues matter via composition, not just presence.

The LSTM sees the sentence as an ordered token sequence, so it can:
model local composition (e.g., “not good”, “beat expectations”, “missed guidance”)
use context to disambiguate words whose sentiment depends on neighbors (e.g., “risk”, “loss”, “decline” vs “loss narrowed”)
represent contrast structure (“X improved, but Y weakened”), which mean-pooling can’t express.

Yes, by a lot. From the learning curves:
BERT val macro-F1 peaks around 0.84
GPT val macro-F1 reaches 0.80
This is a clear gap above the FastText baselines (0.71 for LSTM/MLP).
Pretraining gives domain-agnostic language understanding: syntax, negation, entailment, and sentiment cues are already encoded before seeing your dataset. BERT/GPT produce different vectors for the same word in different contexts (“fall” in “fall in costs” vs “fall in revenue”).

Rank(from best to worst): BERT, GPT, GRU, LSTM, MLP, RNN
Contextual pretraining (BERT/GPT) > gated sequence modeling (GRU/LSTM) > orderless pooling (MLP) > ungated recurrence (RNN).

## AI Use Disclosure (Required)

If you used any AI-enabled tools (e.g., ChatGPT, GitHub Copilot, Claude, or other LLM assistants) while working on this assignment, you must disclose that use here. The goal is transparency-not punishment.

In your disclosure, briefly include:
- **Tool(s) used:** (name + version if known)
- **How you used them:** (e.g., concept explanation, debugging, drafting code, rewriting text)
- **What you verified yourself:** (e.g., reran the notebook, checked outputs/plots, checked shapes, read documentation)
- **What you did *not* use AI for (if applicable):** (optional)

You are responsible for the correctness of your submission, even if AI suggested code or explanations.

#### <font color="red">Write your disclosure here.</font>


Tool used: ChatGPT 5.2 Thinking.
How you used them: polshing sentences, define code structures, debug.
What you verified yourself: ran the code, interpreted the outpluts and plots, read documentation.